# Current Testing Battery

In [1]:
import sys
import statistics
import numpy
from itertools import product
from timeit import default_timer as timer
from tqdm import tqdm
sys.path.append("/home/jackson/Research/GillesPy2/")
import gillespy2
from scipy import stats

Successful Import


In [2]:
from gillespy2.basic_ssa_solver import BasicSSASolver
from gillespy2.ssa_c_solver import SSACSolver
from gillespy2.basic_root_solver import BasicRootSolver
from gillespy2.optimized_ssa_solver import OptimizedSSASolver
from gillespy2.example_models import *

In [3]:
modelList = [Example(), Trichloroethylene(), MichaelisMenten(), Schlogl()]

solverList = [BasicSSASolver, SSACSolver, BasicRootSolver, OptimizedSSASolver]
timingList = []


for model, solver in product(modelList, solverList):
    medianList = []
    exterior_stats = []
    for i in tqdm(range(100), desc = f'Model: {model.name}, Solver: {solver.name}'):
        standard_results = model.run(stochkit_home="/home/jackson/StochKit/StochKit")
        start = timer()
        test_results =model.run(solver=solver)
        stop = timer()
        medianList.append(stop-start)
        interior_stats = []
        for species in standard_results[0].keys():
            if solver == SSACSolver or solver == OptimizedSSASolver:
                interior_stats.append(stats.ks_2samp(standard_results[0][species], test_results[0][species])[0])
            else:
                interior_stats.append(stats.ks_2samp(standard_results[0][species], test_results[species])[0])
        exterior_stats.append(statistics.median(interior_stats))
    median = statistics.median(medianList)
    timingList.append([model, solver, median])

Model: Trichloroethylene, Solver: SSACSolver: 100%|██████████| 100/100 [00:47<00:00,  2.09it/s]
Model: Trichloroethylene, Solver: Basic Root Solver: 100%|██████████| 100/100 [02:38<00:00,  1.58s/it]
Model: Trichloroethylene, Solver: OptimizedSSASolver: 100%|██████████| 100/100 [00:03<00:00, 30.59it/s]
Model: Michaelis_Menten, Solver: SSACSolver: 100%|██████████| 100/100 [00:49<00:00,  2.01it/s]
Model: Michaelis_Menten, Solver: Basic Root Solver: 100%|██████████| 100/100 [04:48<00:00,  2.89s/it]
Model: Michaelis_Menten, Solver: OptimizedSSASolver: 100%|██████████| 100/100 [00:03<00:00, 29.76it/s]
Model: Schlogl, Solver: OptimizedSSASolver: 100%|██████████| 100/100 [00:02<00:00, 40.39it/s]
